In [1]:
#!/usr/bin/env python

"""Tests for `bond` package."""
import sys
from pathlib import Path
sys.path.append("..")
import os
import pytest
from pkg_resources import resource_filename as pkgrf
import shutil
import bond
import tempfile
import os.path as op
from copy import deepcopy
import base64
from glob import glob

TEST_DATA = pkgrf("bond", "testdata")


data_root = Path(".") / "testdata" / "complete"
#shutil.copytree(TEST_DATA, str(data_root))

In [2]:
!find testdata

testdata
testdata/complete
testdata/complete/README
testdata/complete/sub-01
testdata/complete/sub-01/ses-phdiff
testdata/complete/sub-01/ses-phdiff/dwi
testdata/complete/sub-01/ses-phdiff/dwi/sub-01_ses-phdiff_acq-HASC55AP_dwi.bvec
testdata/complete/sub-01/ses-phdiff/dwi/sub-01_ses-phdiff_acq-HASC55AP_dwi.bval
testdata/complete/sub-01/ses-phdiff/dwi/sub-01_ses-phdiff_acq-HASC55AP_dwi.nii.gz
testdata/complete/sub-01/ses-phdiff/dwi/sub-01_ses-phdiff_acq-HASC55AP_dwi.json
testdata/complete/sub-01/ses-phdiff/func
testdata/complete/sub-01/ses-phdiff/func/sub-01_ses-phdiff_task-rest_bold.nii.gz
testdata/complete/sub-01/ses-phdiff/func/sub-01_ses-phdiff_task-rest_bold.json
testdata/complete/sub-01/ses-phdiff/anat
testdata/complete/sub-01/ses-phdiff/anat/sub-01_ses-phdiff_T1w.json
testdata/complete/sub-01/ses-phdiff/anat/sub-01_ses-phdiff_T1w.nii.gz
testdata/complete/sub-01/ses-phdiff/fmap
testdata/complete/sub-01/ses-phdiff/fmap/sub-01_ses-phdiff_dir-PA_epi.json
testdata/complete/sub-01/ses-

In [3]:
from bond import BOnD

bod = BOnD(str(data_root))
key_groups = bod.get_key_groups()
print(key_groups)

['acquisition-HASC55AP_datatype-dwi_suffix-dwi', 'acquisition-v4_datatype-fmap_fmap-magnitude1_suffix-magnitude1', 'acquisition-v4_datatype-fmap_fmap-magnitude2_suffix-magnitude2', 'acquisition-v4_datatype-fmap_fmap-phasediff_suffix-phasediff', 'datatype-anat_suffix-T1w', 'datatype-fmap_direction-PA_fmap-epi_suffix-epi', 'datatype-func_suffix-bold_task-rest']


In [4]:
for key_group in key_groups:
    print(key_group)
    print(bod.get_param_groups(key_group))

acquisition-HASC55AP_datatype-dwi_suffix-dwi
   EffectiveEchoSpacing  EchoTime PhaseEncodingDirection  TotalReadoutTime  \
0               0.00069     0.089                      j           0.07176   

   FlipAngle  RepetitionTime  SliceTime000  SliceTime001  SliceTime002  \
0         90             4.2           0.0      0.116667      0.233333   

   SliceTime003  ...  SliceTime026  SliceTime027  SliceTime028  SliceTime029  \
0          0.35  ...      3.033333          3.15      3.266667      3.383333   

   SliceTime030  SliceTime031  SliceTime032  SliceTime033  SliceTime034  \
0           3.5      3.616667      3.733333          3.85      3.966667   

   SliceTime035  
0      4.083333  

[1 rows x 42 columns]
acquisition-v4_datatype-fmap_fmap-magnitude1_suffix-magnitude1
   EchoTime  PartialFourier  FlipAngle PhaseEncodingDirection  RepetitionTime
0     0.004            0.75         60                     j-             1.5
acquisition-v4_datatype-fmap_fmap-magnitude2_suffix-magnitu

In [5]:
data_root = Path(".") / "testdata" / "inconsistent"
multigroup_bod = BOnD(str(data_root))
key_groups = multigroup_bod.get_key_groups()
print(key_groups)

['acquisition-HASC55AP_datatype-dwi_suffix-dwi', 'acquisition-v4_datatype-fmap_fmap-magnitude1_suffix-magnitude1', 'acquisition-v4_datatype-fmap_fmap-magnitude2_suffix-magnitude2', 'acquisition-v4_datatype-fmap_fmap-phasediff_suffix-phasediff', 'datatype-anat_suffix-T1w', 'datatype-fmap_direction-PA_fmap-epi_suffix-epi', 'datatype-func_suffix-bold_task-rest']


In [6]:
for key_group in key_groups:
    print(key_group)
    print(multigroup_bod.get_param_groups(key_group))

acquisition-HASC55AP_datatype-dwi_suffix-dwi
   EffectiveEchoSpacing  EchoTime PhaseEncodingDirection  TotalReadoutTime  \
0               0.00069     0.040                      j           0.07176   
1               0.00069     0.089                      j           0.07176   

   FlipAngle  RepetitionTime  SliceTime000  SliceTime001  SliceTime002  \
0         90             4.2           0.0      0.116667      0.233333   
1         90             4.2           0.0      0.116667      0.233333   

   SliceTime003  ...  SliceTime026  SliceTime027  SliceTime028  SliceTime029  \
0          0.35  ...      3.033333          3.15      3.266667      3.383333   
1          0.35  ...      3.033333          3.15      3.266667      3.383333   

   SliceTime030  SliceTime031  SliceTime032  SliceTime033  SliceTime034  \
0           3.5      3.616667      3.733333          3.85      3.966667   
1           3.5      3.616667      3.733333          3.85      3.966667   

   SliceTime035  
0      4.083